<a href="https://colab.research.google.com/github/MilindAP/Multi-Link-Prediction-for-mmWave-and-subTHz-frequencies-using-LTC-LSTM-and-Symbolic-Regression/blob/main/Multi_Link_Prediction_LTC_Based_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
Data1 = pd.read_csv("/content/drive/MyDrive/My docs/MultiLink Prediction/data.csv")
#Data1 = Data['SNR']
#Data1

In [3]:
Data1.to_csv('Data1.csv')
Data1.reset_index(drop=True, inplace=True)

In [4]:
Data1

,Received Power (dBm),RMS Delay Spread (ns),SNR,NA4,NA5,NA6,NA7,NA8,NA9,NA10,NA11,NA12,NA13,NA14,NA15,NA16,NA17,NA18
0,-77.5,8.8,-8.611111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-89.2,3.0,-9.911111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-89.4,11.9,-9.933333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-89.5,12.1,-9.944444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-81.3,3.9,-9.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2832,-167.0,31.9,-18.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2833,-167.0,4.4,-18.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2834,-167.0,4.3,-18.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2835,-167.0,13.3,-18.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import tensorflow as tf
import numpy as np
import time
import os
from enum import Enum

class MappingType(Enum):
    Identity = 0
    Linear = 1
    Affine = 2

class ODESolver(Enum):
    SemiImplicit = 0
    Explicit = 1
    RungeKutta = 2

class LTCCell(tf.compat.v1.nn.rnn_cell.RNNCell):

    def __init__(self, num_units):

        self._input_size = -1
        self._num_units = num_units
        self._is_built = False

        # Number of ODE solver steps in one RNN step
        self._ode_solver_unfolds = 6
        self._solver = ODESolver.SemiImplicit

        self._input_mapping = MappingType.Affine

        self._erev_init_factor = 1

        self._w_init_max = 1.0
        self._w_init_min = 0.01
        self._cm_init_min = 0.5
        self._cm_init_max = 0.5
        self._gleak_init_min = 1
        self._gleak_init_max = 1

        self._w_min_value = 0.00001
        self._w_max_value = 1000
        self._gleak_min_value = 0.00001
        self._gleak_max_value = 1000
        self._cm_t_min_value = 0.000001
        self._cm_t_max_value = 1000

        self._fix_cm = None
        self._fix_gleak = None
        self._fix_vleak = None

    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def _map_inputs(self,inputs,resuse_scope=False):
        varscope = "sensory_mapping"
        reuse = tf.compat.v1.AUTO_REUSE
        if(resuse_scope):
            varscope = self._sensory_varscope
            reuse = True
            tf.reset_default_graph()

        with tf.compat.v1.variable_scope(varscope,reuse=reuse) as scope:
            self._sensory_varscope = scope
            if(self._input_mapping == MappingType.Affine or self._input_mapping == MappingType.Linear):
                w =  tf.compat.v1.get_variable(name='input_w',shape=[self._input_size],trainable=True,initializer=tf.initializers.constant(1),reuse=True)
                inputs = inputs * w
            if(self._input_mapping == MappingType.Affine):
                b =  tf.compat.v1.get_variable(name='input_b',shape=[self._input_size],trainable=True,initializer=tf.initializers.constant(0),reuse=True)
                inputs = inputs + b
        return inputs

    # TODO: Implement RNNLayer properly,i.e, allocate variables here
    def build(self,input_shape):
        pass

    def __call__(self, inputs, state, scope=None):
        with tf.compat.v1.variable_scope("ltc"):
            if(not self._is_built):
                # TODO: Move this part into the build method inherited form tf.Layers
                self._is_built = True
                self._input_size = int(inputs.shape[-1])

                self._get_variables()

            elif(self._input_size != int(inputs.shape[-1])):
                raise ValueError("You first feed an input with {} features and now one with {} features, that is not possible".format(
                    self._input_size,
                    int(inputs[-1])
                ))

            inputs = self._map_inputs(inputs)

            if(self._solver == ODESolver.Explicit):
                next_state = self._ode_step_explicit(inputs,state,_ode_solver_unfolds=self._ode_solver_unfolds)
            elif(self._solver == ODESolver.SemiImplicit):
                next_state = self._ode_step(inputs,state)
            elif(self._solver == ODESolver.RungeKutta):
                next_state = self._ode_step_runge_kutta(inputs,state)
            else:
                raise ValueError("Unknown ODE solver '{}'".format(str(self._solver)))

            outputs = next_state

        return outputs, next_state

    # Create tf variables
    def _get_variables(self):
        self.sensory_mu = tf.compat.v1.get_variable(name='sensory_mu',shape=[self._input_size,self._num_units],trainable=True,initializer=tf.initializers.random_uniform(minval=0.3,maxval=0.8))
        self.sensory_sigma = tf.compat.v1.get_variable(name='sensory_sigma',shape=[self._input_size,self._num_units],trainable=True,initializer=tf.initializers.random_uniform(minval=3.0,maxval=8.0))
        self.sensory_W = tf.compat.v1.get_variable(name='sensory_W',shape=[self._input_size,self._num_units],trainable=True,initializer=tf.initializers.constant(np.random.uniform(low=self._w_init_min,high=self._w_init_max,size=[self._input_size,self._num_units])))
        sensory_erev_init = 2*np.random.randint(low=0,high=2,size=[self._input_size,self._num_units])-1
        self.sensory_erev = tf.compat.v1.get_variable(name='sensory_erev',shape=[self._input_size,self._num_units],trainable=True,initializer=tf.initializers.constant(sensory_erev_init*self._erev_init_factor))

        self.mu = tf.compat.v1.get_variable(name='mu',shape=[self._num_units,self._num_units],trainable=True,initializer=tf.initializers.random_uniform(minval=0.3,maxval=0.8))
        self.sigma = tf.compat.v1.get_variable(name='sigma',shape=[self._num_units,self._num_units],trainable=True,initializer=tf.initializers.random_uniform(minval=3.0,maxval=8.0))
        self.W = tf.compat.v1.get_variable(name='W',shape=[self._num_units,self._num_units],trainable=True,initializer=tf.initializers.constant(np.random.uniform(low=self._w_init_min,high=self._w_init_max,size=[self._num_units,self._num_units])))

        erev_init = 2*np.random.randint(low=0,high=2,size=[self._num_units,self._num_units])-1
        self.erev = tf.compat.v1.get_variable(name='erev',shape=[self._num_units,self._num_units],trainable=True,initializer=tf.initializers.constant(erev_init*self._erev_init_factor))

        if(self._fix_vleak is None):
            self.vleak = tf.compat.v1.get_variable(name='vleak',shape=[self._num_units],trainable=True,initializer=tf.initializers.random_uniform(minval=-0.2,maxval=0.2))
        else:
            self.vleak = tf.compat.v1.get_variable(name='vleak',shape=[self._num_units],trainable=False,initializer=tf.initializers.constant(self._fix_vleak))

        if(self._fix_gleak is None):
            initializer=tf.initializers.constant(self._gleak_init_min)
            if(self._gleak_init_max > self._gleak_init_min):
                initializer = tf.initializers.random_uniform(minval= self._gleak_init_min,maxval = self._gleak_init_max)
            self.gleak = tf.compat.v1.get_variable(name='gleak',shape=[self._num_units],trainable=True,initializer=initializer)
        else:
            self.gleak = tf.compat.v1.get_variable(name='gleak',shape=[self._num_units],trainable=False,initializer=tf.initializers.constant(self._fix_gleak))

        if(self._fix_cm is None):
            initializer=tf.initializers.constant(self._cm_init_min)
            if(self._cm_init_max > self._cm_init_min):
                initializer = tf.initializers.random_uniform(minval= self._cm_init_min,maxval = self._cm_init_max)
            self.cm_t = tf.compat.v1.get_variable(name='cm_t',shape=[self._num_units],trainable=True,initializer=initializer)
        else:
            self.cm_t = tf.compat.v1.get_variable(name='cm_t',shape=[self._num_units],trainable=False,initializer=tf.initializers.constant(self._fix_cm))

    # Hybrid euler method
    def _ode_step(self,inputs,state):
        v_pre = state

        sensory_w_activation = self.sensory_W*self._sigmoid(inputs,self.sensory_mu,self.sensory_sigma)
        sensory_rev_activation = sensory_w_activation*self.sensory_erev

        w_numerator_sensory = tf.reduce_sum(sensory_rev_activation,axis=1)
        w_denominator_sensory = tf.reduce_sum(sensory_w_activation,axis=1)

        for t in range(self._ode_solver_unfolds):
            w_activation = self.W*self._sigmoid(v_pre,self.mu,self.sigma)

            rev_activation = w_activation*self.erev

            w_numerator = tf.reduce_sum(rev_activation,axis=1) + w_numerator_sensory
            w_denominator = tf.reduce_sum(w_activation,axis=1) + w_denominator_sensory

            numerator = self.cm_t * v_pre + self.gleak*self.vleak + w_numerator
            denominator = self.cm_t + self.gleak + w_denominator

            v_pre = numerator/denominator

        return v_pre

    def _f_prime(self,inputs,state):
        v_pre = state

        # We can pre-compute the effects of the sensory neurons here
        sensory_w_activation = self.sensory_W*self._sigmoid(inputs,self.sensory_mu,self.sensory_sigma)
        w_reduced_sensory = tf.reduce_sum(sensory_w_activation,axis=1)

        # Unfold the mutliply ODE multiple times into one RNN step
        w_activation = self.W*self._sigmoid(v_pre,self.mu,self.sigma)

        w_reduced_synapse = tf.reduce_sum(w_activation,axis=1)

        sensory_in = self.sensory_erev * sensory_w_activation
        synapse_in = self.erev * w_activation

        sum_in = tf.reduce_sum(sensory_in,axis=1) - v_pre*w_reduced_synapse + tf.reduce_sum(synapse_in,axis=1) - v_pre * w_reduced_sensory

        f_prime = 1/self.cm_t * (self.gleak * (self.vleak-v_pre) + sum_in)

        return f_prime

    def _ode_step_runge_kutta(self,inputs,state):

        h = 0.1
        for i in range(self._ode_solver_unfolds):
            k1 = h*self._f_prime(inputs,state)
            k2 = h*self._f_prime(inputs,state+k1*0.5)
            k3 = h*self._f_prime(inputs,state+k2*0.5)
            k4 = h*self._f_prime(inputs,state+k3)

            state = state + 1.0/6*(k1+2*k2+2*k3+k4)

        return state

    def _ode_step_explicit(self,inputs,state,_ode_solver_unfolds):
        v_pre = state

        # We can pre-compute the effects of the sensory neurons here
        sensory_w_activation = self.sensory_W*self._sigmoid(inputs,self.sensory_mu,self.sensory_sigma)
        w_reduced_sensory = tf.reduce_sum(sensory_w_activation,axis=1)


        # Unfold the mutliply ODE multiple times into one RNN step
        for t in range(_ode_solver_unfolds):
            w_activation = self.W*self._sigmoid(v_pre,self.mu,self.sigma)

            w_reduced_synapse = tf.reduce_sum(w_activation,axis=1)

            sensory_in = self.sensory_erev * sensory_w_activation
            synapse_in = self.erev * w_activation

            sum_in = tf.reduce_sum(sensory_in,axis=1) - v_pre*w_reduced_synapse + tf.reduce_sum(synapse_in,axis=1) - v_pre * w_reduced_sensory

            f_prime = 1/self.cm_t * (self.gleak * (self.vleak-v_pre) + sum_in)

            v_pre = v_pre + 0.1 * f_prime

        return v_pre

    def _sigmoid(self,v_pre,mu,sigma):
        v_pre = tf.reshape(v_pre,[-1,v_pre.shape[-1],1])
        mues = v_pre - mu
        x = sigma*mues
        return tf.nn.sigmoid(x)

    def get_param_constrain_op(self):

        cm_clipping_op = tf.compat.v1.assign(self.cm_t,tf.clip_by_value(self.cm_t, self._cm_t_min_value, self._cm_t_max_value))
        gleak_clipping_op = tf.compat.v1.assign(self.gleak,tf.clip_by_value(self.gleak, self._gleak_min_value, self._gleak_max_value))
        w_clipping_op = tf.compat.v1.assign(self.W,tf.clip_by_value(self.W, self._w_min_value, self._w_max_value))
        sensory_w_clipping_op = tf.compat.v1.assign(self.sensory_W ,tf.clip_by_value(self.sensory_W, self._w_min_value, self._w_max_value))

        return [cm_clipping_op,gleak_clipping_op,w_clipping_op,sensory_w_clipping_op]

    def export_weights(self,dirname,sess,output_weights=None):
        os.makedirs(dirname,exist_ok=True)
        w,erev,mu,sigma = sess.run([self.W,self.erev,self.mu,self.sigma])
        sensory_w,sensory_erev,sensory_mu,sensory_sigma = sess.run([self.sensory_W,self.sensory_erev,self.sensory_mu,self.sensory_sigma])
        vleak,gleak,cm = sess.run([self.vleak,self.gleak,self.cm_t])

        if(not output_weights is None):
            output_w,output_b = sess.run(output_weights)
            np.savetxt(os.path.join(dirname,"output_w.csv"),output_w)
            np.savetxt(os.path.join(dirname,"output_b.csv"),output_b)
        np.savetxt(os.path.join(dirname,"w.csv"),w)
        np.savetxt(os.path.join(dirname,"erev.csv"),erev)
        np.savetxt(os.path.join(dirname,"mu.csv"),mu)
        np.savetxt(os.path.join(dirname,"sigma.csv"),sigma)
        np.savetxt(os.path.join(dirname,"sensory_w.csv"),sensory_w)
        np.savetxt(os.path.join(dirname,"sensory_erev.csv"),sensory_erev)
        np.savetxt(os.path.join(dirname,"sensory_mu.csv"),sensory_mu)
        np.savetxt(os.path.join(dirname,"sensory_sigma.csv"),sensory_sigma)
        np.savetxt(os.path.join(dirname,"vleak.csv"),vleak)
        np.savetxt(os.path.join(dirname,"gleak.csv"),gleak)
        np.savetxt(os.path.join(dirname,"cm.csv"),cm)

In [6]:
import tensorflow as tf
import numpy  as np
import os


class CTRNN(tf.compat.v1.nn.rnn_cell.RNNCell):

    def __init__(self, num_units,cell_clip=-1,global_feedback=False,fix_tau=True):
        self._num_units = num_units
        # Number of ODE solver steps
        self._unfolds = 6
        # Time of each ODE solver step, for variable time RNN change this
        # to a placeholder/non-trainable variable
        self._delta_t = 0.1

        self.global_feedback = global_feedback

        # Time-constant of the cell
        self.fix_tau = fix_tau
        self.tau = 1
        self.cell_clip = cell_clip


    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def export_weights(self,dirname,sess,output_weights=None):
        os.makedirs(dirname,exist_ok=True)
        w,b = sess.run([self.W,self.b])

        tau = np.ones(1)
        if(not self.fix_tau):
            sp_op = tf.nn.softplus(self._tau_var)
            tau = sess.run(sp_op)
        if(not output_weights is None):
            output_w,output_b = sess.run(output_weights)
            np.savetxt(os.path.join(dirname,"output_w.csv"),output_w)
            np.savetxt(os.path.join(dirname,"output_b.csv"),output_b)
        np.savetxt(os.path.join(dirname,"w.csv"),w)
        np.savetxt(os.path.join(dirname,"b.csv"),b)
        np.savetxt(os.path.join(dirname,"tau.csv"),b)

    # TODO: Implement RNNLayer properly,i.e, allocate variables here
    def build(self,input_shape):
        pass

    def _dense(self,units,inputs,activation,name,bias_initializer=tf.constant_initializer(0.0)):
        input_size = int(inputs.shape[-1])
        W = tf.get_variable('W_{}'.format(name), [input_size, units])
        b = tf.get_variable("bias_{}".format(name), [units],initializer=bias_initializer)

        y = tf.matmul(inputs,W) + b
        if(not activation is None):
            y = activation(y)

        self.W = W
        self.b = b

        return y

    def __call__(self, inputs, state, scope=None):
        # CTRNN ODE is: df/dt = NN(x) - f
        # where x is the input, and NN is a MLP.
        # Input could be: 1: just the input of the RNN cell
        # or 2: input of the RNN cell merged with the current state

        self._input_size = int(inputs.shape[-1])
        with tf.variable_scope(scope or type(self).__name__):
          #with tf.variable_scope('lstm', reuse=True) as scope:
            with tf.variable_scope("rnn",reuse=tf.AUTO_REUSE):  # Reset gate and update gate.
                if(not self.fix_tau):
                    tau = tf.get_variable('tau', [],initializer=tf.constant_initializer(self.tau))
                    self._tau_var = tau
                    tau = tf.nn.softplus(tau) # Make sure tau is positive
                else:
                    tau = self.tau

                # Input Option 1: RNNCell input
                if(not self.global_feedback):
                    input_f_prime = self._dense(units=self._num_units,inputs=inputs,activation=tf.nn.tanh,name="step")
                for i in range(self._unfolds):
                    # Input Option 2: RNNCell input AND RNN state
                    if(self.global_feedback):
                        fused_input = tf.concat([inputs,state],axis=-1)
                        input_f_prime = self._dense(units=self._num_units,inputs=fused_input,activation=tf.nn.tanh,name="step")

                    # df/dt
                    f_prime = -state/self.tau + input_f_prime

                    # If we solve this ODE with explicit euler we get
                    # f(t+deltaT) = f(t) + deltaT * df/dt
                    state = state + self._delta_t * f_prime

                    # Optional clipping of the RNN cell to enforce stability (not needed)
                    if(self.cell_clip > 0):
                        state = tf.clip_by_value(state,-self.cell_clip,self.cell_clip)

        return state,state


class NODE(tf.compat.v1.nn.rnn_cell.RNNCell):

    def __init__(self, num_units,cell_clip=-1):
        self._num_units = num_units
        # Number of ODE solver steps
        self._unfolds = 6
        # Time of each ODE solver step, for variable time RNN change this
        # to a placeholder/non-trainable variable
        self._delta_t = 0.1

        self.cell_clip = cell_clip


    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def export_weights(self,dirname,sess,output_weights=None):
        os.makedirs(dirname,exist_ok=True)
        w,b = sess.run([self.W,self.b])

        tau = np.ones(1)
        if(not self.fix_tau):
            sp_op = tf.nn.softplus(self._tau_var)
            tau = sess.run(sp_op)
        if(not output_weights is None):
            output_w,output_b = sess.run(output_weights)
            np.savetxt(os.path.join(dirname,"output_w.csv"),output_w)
            np.savetxt(os.path.join(dirname,"output_b.csv"),output_b)
        np.savetxt(os.path.join(dirname,"w.csv"),w)
        np.savetxt(os.path.join(dirname,"b.csv"),b)
        np.savetxt(os.path.join(dirname,"tau.csv"),b)

    # TODO: Implement RNNLayer properly,i.e, allocate variables here
    def build(self,input_shape):
        pass


    def _ode_step_runge_kutta(self,inputs,state):

        for i in range(self._unfolds):
            k1 = self._delta_t*self._f_prime(inputs,state)
            k2 = self._delta_t*self._f_prime(inputs,state+k1*0.5)
            k3 = self._delta_t*self._f_prime(inputs,state+k2*0.5)
            k4 = self._delta_t*self._f_prime(inputs,state+k3)

            state = state + (k1+2*k2+2*k3+k4)/6.0

            # Optional clipping of the RNN cell to enforce stability (not needed)
            if(self.cell_clip > 0):
                state = tf.clip_by_value(state,-self.cell_clip,self.cell_clip)

        return state

    def _f_prime(self,inputs,state):
        fused_input = tf.concat([inputs,state],axis=-1)
        input_f_prime = self._dense(units=self._num_units,inputs=fused_input,activation=tf.nn.tanh,name="step")
        return input_f_prime

    def _dense(self,units,inputs,activation,name,bias_initializer=tf.constant_initializer(0.0)):
        input_size = int(inputs.shape[-1])
        W = tf.get_variable('W_{}'.format(name), [input_size, units])
        b = tf.get_variable("bias_{}".format(name), [units],initializer=bias_initializer)

        y = tf.matmul(inputs,W) + b
        if(not activation is None):
            y = activation(y)

        self.W = W
        self.b = b

        return y

    def __call__(self, inputs, state, scope=None):
        # CTRNN ODE is: df/dt = NN(x) - f
        # where x is the input, and NN is a MLP.
        # Input could be: 1: just the input of the RNN cell
        # or 2: input of the RNN cell merged with the current state

        self._input_size = int(inputs.shape[-1])
        with tf.variable_scope(scope or type(self).__name__):
            with tf.variable_scope("RNN",reuse=tf.AUTO_REUSE):  # Reset gate and update gate.

                state = self._ode_step_runge_kutta(inputs,state)

        return state,state



class CTGRU(tf.compat.v1.nn.rnn_cell.RNNCell):
    # https://arxiv.org/abs/1710.04110
    def __init__(self, num_units,M=8,cell_clip=-1):
        self._num_units = num_units
        self.M = M
        self.cell_clip = cell_clip
        self.ln_tau_table = np.empty(self.M)
        tau = 1
        for i in range(self.M):
            self.ln_tau_table[i] = np.log(tau)
            tau = tau * (10.0**0.5)

    @property
    def state_size(self):
        return self._num_units*self.M

    @property
    def output_size(self):
        return self._num_units

    # TODO: Implement RNNLayer properly,i.e, allocate variables here
    def build(self,input_shape):
        pass

    def _dense(self,units,inputs,activation,name,bias_initializer=tf.constant_initializer(0.0)):
        input_size = int(inputs.shape[-1])
        W = tf.get_variable('W_{}'.format(name), [input_size, units])
        b = tf.get_variable("bias_{}".format(name), [units],initializer=bias_initializer)

        y = tf.matmul(inputs,W) + b
        if(not activation is None):
            y = activation(y)

        return y

    def __call__(self, inputs, state, scope=None):
        self._input_size = int(inputs.shape[1])

        # CT-GRU input is actually a matrix and not a vector
        h_hat = tf.reshape(state,[-1,self._num_units,self.M])
        h = tf.reduce_sum(h_hat,axis=2)
        state = None # Set state to None, to avoid misuses (bugs) in the code below

        with tf.variable_scope(scope or type(self).__name__):
            with tf.variable_scope("Gates"):  # Reset gate and update gate.
                fused_input = tf.concat([inputs,h],axis=-1)
                ln_tau_r = tf.layers.Dense(self._num_units*self.M,activation=None,name="tau_r")(fused_input)
                ln_tau_r = tf.reshape(ln_tau_r,shape=[-1,self._num_units,self.M])
                sf_input_r = -tf.square(ln_tau_r-self.ln_tau_table)
                rki = tf.nn.softmax(logits=sf_input_r,axis=2)

                q_input = tf.reduce_sum(rki*h_hat,axis=2)
                reset_value = tf.concat([inputs,q_input],axis=1)
                qk = self._dense(units=self._num_units,inputs=reset_value,activation=tf.nn.tanh,name="detect_signal")

                qk = tf.reshape(qk,[-1,self._num_units,1]) # in order to broadcast

                ln_tau_s = tf.layers.Dense(self._num_units*self.M,activation=None,name="tau_s")(fused_input)
                ln_tau_s = tf.reshape(ln_tau_s,shape=[-1,self._num_units,self.M])
                sf_input_s = -tf.square(ln_tau_s-self.ln_tau_table)
                ski = tf.nn.softmax(logits=sf_input_s,axis=2)

                h_hat_next = ((1-ski)*h_hat + ski*qk)*np.exp(-1.0/self.ln_tau_table)

                if(self.cell_clip > 0):
                    h_hat_next = tf.clip_by_value(h_hat_next,-self.cell_clip,self.cell_clip)
                # Compute new state
                h_next = tf.reduce_sum(h_hat_next,axis=2)
                h_hat_next_flat = tf.reshape(h_hat_next,shape=[-1,self._num_units*self.M])

        return h_next, h_hat_next_flat

In [7]:
import numpy as np
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Run on CPU

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
#import ltc_model as ltc
#from ctrnn_model import CTRNN, NODE, CTGRU
import argparse
import datetime as dt
#import keras


def cut_in_sequences(x,seq_len,inc=1):
    # given a file, produces sequences of length seq_len incrementing with inc. Basically generates sequences of data
    sequences_x = []
    sequences_y = []

    for s in range(0,x.shape[0] - seq_len-1,inc):
        start = s
        end = start+seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(x[start+1:end+1])

    return sequences_x,sequences_y

class ENSO_data:

    def __init__(self,seq_len=32):

        data1 = pd.read_csv("/content/Data1.csv")
        #data = pd.read_csv("D:\Reasearch projects\LTC for ENSO\data\ENSO.csv")
        #data1 = np.array(data).reshape(-1, 1)
        data_processed = self.process_data(data1)
        #self.train_x,self.test_x,self.valid_x,self.train_y,self.test_y,self.valid_y = self.split_data(data_processed)

        self.train_x,self.train_y = self.load_data(data_processed[:1250, :])
        self.test_x,self.test_y = self.load_data(data_processed[1251:2350, :])
        self.valid_x,self.valid_y = self.load_data(data_processed[2351:, :])



        #self.train_x = self.train_x.reshape(self.train_x.shape[0], self.train_x.shape[1])
        #self.test_x = self.test_x.reshape(self.test_x.shape[0], self.test_x.shape[1])
        #self.valid_x = self.valid_x.reshape(self.valid_x.shape[0],self.valid_x.shape[1])


        self.seq_len = seq_len
        self.obs_size = 17

    def load_data(self,data): #loads npy files.
        all_x = []
        all_y = []


        #r = r.astype(np.float32)
        x,y = cut_in_sequences(data,32,10)

        all_x.extend(x)
        all_y.extend(y)

        test_x = np.stack(all_x,axis=1)
        test_y = np.stack(all_y,axis=1)

        return np.stack(all_x,axis=1),np.stack(all_y,axis=1)



    def split_data(self,data):
        #values = data.values
        x_train, x_test,x_val,y_train, y_test,y_val = data[:1250, :-1], data[1251:2350, :-1],data[2351:, :-1], data[:1250, -1:], data[1251:2350, -1:],data[2351:, -1:]
        return  np.array(x_train), np.array(x_test),np.array(x_val),np.array(y_train), np.array(y_test),np.array(y_val)


    def iterate_train(self,batch_size=20):
        total_seqs = self.train_x.shape[0]
        permutation = np.random.permutation(total_seqs)
        total_batches = total_seqs // batch_size

        for i in range(total_batches):
            start = i*batch_size
            end = start + batch_size
            batch_x = self.train_x[permutation[start:end], :]
            batch_y = self.train_y[permutation[start:end], :]
            yield (batch_x,batch_y)

    def series_to_supervised(self,data, n_in=1, n_out=1, dropnan=True):

        n_vars = 1 if type(data) is list else data.shape[1]
        df = pd.DataFrame(data)
        cols, names = list(), list()
        # input sequence (t-n, ... t-1)
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var{}(t-{})'.format(j+1, i)) for j in range(n_vars)]
        # forecast sequence (t, t+1, ... t+n)
        for i in range(0, n_out):
            cols.append(df.shift(-i).iloc[:, -1])
            if i == 0:
                names += ['var(t)']
            else:
                names += ['var(t+{})'.format(i)]
        # put it all together
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # drop rows with NaN values
        if dropnan:
            agg.dropna(inplace=True)
        return agg

    def process_data(self,data1):
        #data1.replace('NAN', np.nan, inplace=True)
        #data1.isna().sum(axis=0)
        #data.rename(columns={'Precipitation (mm/day)': 'Precipitation'}, inplace=True)
        #data1 = [str(i).replace(",", "") for i in data1]
        data1 = data1.astype({'SNR': float})
        '''data1 = data1.astype({
                'Month': str,
                'Season': str,
                'SNR': float,
                'NINO 1+2 SST': float,
                'NINO 1+2 SST Anomalies': float,
                'NINO 3 SST': float,
                'NINO 3 SST Anomalies': float,
                'NINO 3.4 SST': float,
                'NINO 3.4 SST Anomalies': float,
                'NINO 4 SST': float,
                'NINO 4 SST Anomalies': float,
                'OLR': float,
                'TNI': float,
                'Precipitation': float
             })'''

        #mean_month = data.groupby('Month')[data.columns[4:]].transform('mean')
        #data.fillna(mean_month, inplace=True)
        #data.set_index('Date', inplace= True)
        #indicators = ['ONI', 'NINO 3.4 SST Anomalies', 'OLR', 'TNI', 'PNA', 'Precipitation', 'SOI'] #from correlation matrix
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler(feature_range = (0,1))
        df_scaled = scaler.fit_transform(data1)
        df_transformed = df_scaled
        #df_transformed = self.series_to_supervised(df_scaled, 1, 1)
        return df_transformed

class ENSO_model:

    def __init__(self,model_type,model_size,sparsity_level=0.5,learning_rate = 0.001):
        self.model_type = model_type
        self.constrain_op = []
        self.sparsity_level = sparsity_level
        self.x = tf.compat.v1.placeholder(dtype=tf.float32,shape=[None, None, 2])
        #self.x = tf.compat.v1.placeholder(dtype=tf.float32, shape=[32, 107, 2])
        self.target_y = tf.compat.v1.placeholder(dtype=tf.float32,shape=[None, None, 2])
        #self.target_y = tf.compat.v1.placeholder(dtype=tf.float32, shape=[32, 107, 2])

        self.model_size = model_size
        head = self.x

        if(model_type == "lstm"):
            self.fused_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(model_size)
            #tf.compat.v1.reset_default_graph()

            head,_ = tf.compat.v1.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        elif(model_type.startswith("ltc")):
            learning_rate = 0.01 # LTC needs a higher learning rate
            self.wm = ltc.LTCCell(model_size)
            if(model_type.endswith("_rk")):
                self.wm._solver = ltc.ODESolver.RungeKutta
            elif(model_type.endswith("_ex")):
                self.wm._solver = ltc.ODESolver.Explicit
            else:
                self.wm._solver = ltc.ODESolver.SemiImplicit

            head,_ = tf.compat.v1.nn.dynamic_rnn(self.wm,head,dtype=tf.float32,time_major=True)
            self.constrain_op.extend(self.wm.get_param_constrain_op())
        elif(model_type == "node"):
            self.fused_cell = NODE(model_size,cell_clip=-1)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        elif(model_type == "ctgru"):
            self.fused_cell = CTGRU(model_size,cell_clip=-1)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        elif(model_type == "ctrnn"):
            self.fused_cell = CTRNN(model_size,cell_clip=-1,global_feedback=True)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        else:
            raise ValueError("Unknown model type '{}'".format(model_type))

        if(self.sparsity_level > 0):
            self.constrain_op.extend(self.get_sparsity_ops())

        self.y = tf.compat.v1.layers.dense(head,units=1,activation=None)
        #self.y = tf.keras.layers.Dense(head,units=1,activation=None)
        print("logit shape: ",str(self.y.shape))
        self.loss = tf.reduce_mean(tf.square(self.target_y-self.y))
        self.rmse = tf.sqrt(self.loss)
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
        self.train_step = optimizer.minimize(self.loss)

        self.accuracy = tf.reduce_mean(tf.abs(self.target_y-self.y))

        self.sess = tf.compat.v1.InteractiveSession()
        self.sess.run(tf.compat.v1.global_variables_initializer())

        self.result_file = os.path.join("results","Data1","{}_{}_{:02d}.csv".format(model_type,model_size,int(100*self.sparsity_level)))
        if(not os.path.exists("/content/results/Data1")):
            os.makedirs("/content/results/Data1")
        if(not os.path.isfile(self.result_file)):
            with open(self.result_file,"w") as f:
                f.write("best epoch, train loss, train mae, valid loss, valid mae,valid rmse, test loss, test mae,test rmse\n")

        self.checkpoint_path = os.path.join("tf_sessions","cheetah","{}".format(model_type))
        if(not os.path.exists("tf_sessions/cheetah")):
            os.makedirs("tf_sessions/cheetah")

        self.saver = tf.compat.v1.train.Saver()


    def get_sparsity_ops(self):
        tf_vars = tf.trainable_variables()
        op_list = []
        for v in tf_vars:
            #print("Variable {}".format(str(v)))
            if(v.name.startswith("rnn")):
                if(len(v.shape)<2):
                    # Don't sparsity biases
                    continue
                if("ltc" in v.name and (not "W:0" in v.name)):
                    # LTC can be sparsified by only setting w[i,j] to 0
                    # both input and recurrent matrix will be sparsified
                    continue
            op_list.append(self.sparse_var(v,self.sparsity_level))

        return op_list

    def sparse_var(self,v,sparsity_level):
        mask = np.random.choice([0, 1], size=v.shape, p=[sparsity_level,1-sparsity_level]).astype(np.float32)
        v_assign_op = tf.assign(v,v*mask)
        print("Var[{}] will be sparsified with {:0.2f} sparsity level".format(
            v.name,sparsity_level
        ))
        return v_assign_op


    def save(self):
        self.saver.save(self.sess, self.checkpoint_path)

    def restore(self):
        self.saver.restore(self.sess, self.checkpoint_path)


    def fit(self,cheetah_data,epochs,verbose=True,log_period=50):

        best_valid_loss = np.PINF
        best_valid_stats = (0,0,0,0,0,0,0,0,0)
        self.save()
        for e in range(epochs):
            if(verbose and e%log_period == 0):
                test_acc,test_loss,test_rmse = self.sess.run([self.accuracy,self.loss,self.rmse],{self.x:cheetah_data.test_x,self.target_y: cheetah_data.test_y})
                valid_acc,valid_loss,valid_rmse = self.sess.run([self.accuracy,self.loss,self.rmse],{self.x:cheetah_data.valid_x,self.target_y: cheetah_data.valid_y})
                if((valid_loss < best_valid_loss and e > 0) or e==1):
                    # print("BEST")
                    best_valid_loss = valid_loss
                    best_valid_stats = (
                        e,
                        np.mean(losses),np.mean(accs),
                        valid_loss,valid_acc,valid_rmse,
                        test_loss,test_acc,test_rmse)
                    self.save()

            losses = []
            accs = []
            for batch_x,batch_y in cheetah_data.iterate_train(batch_size=10):
                acc,loss,_ = self.sess.run([self.accuracy,self.loss,self.train_step],{self.x:batch_x,self.target_y: batch_y})
                if(len(self.constrain_op) > 0):
                    self.sess.run(self.constrain_op)

                losses.append(loss)
                accs.append(acc)

            if(verbose and e%log_period == 0):
                print("Epochs {:03d}, train loss: {:0.2f}, train mae: {:0.2f}, valid loss: {:0.2f}, valid mae: {:0.2f}, test loss: {:0.2f}, test mae: {:0.2f}".format(
                    e,
                    np.mean(losses),np.mean(accs),
                    valid_loss,valid_acc,valid_rmse,
                    test_loss,test_acc,test_rmse
                ))
            if(e > 0 and (not np.isfinite(np.mean(losses)))):
                break
        self.restore()
        best_epoch,train_loss,train_acc,valid_loss,valid_acc,valid_rmse,test_loss,test_acc,test_rmse = best_valid_stats
        print("Best epoch {:03d}, train loss: {:0.2f}, train mae: {:0.2f}, valid loss: {:0.2f}, valid mae: {:0.2f},valid rmse: {:0.2f}, test loss: {:0.2f}, test mae: {:0.2f},test rmse: {:0.2f}".format(
            best_epoch,
            train_loss,train_acc,
            valid_loss,valid_acc,valid_rmse,
            test_loss,test_acc,test_rmse
        ))
        with open(self.result_file,"a") as f:
            f.write("{:08d}, {:0.8f}, {:0.8f}, {:0.8f}, {:0.8f}, {:0.8f}, {:0.8f},{:0.8f},{:0.8f}\n".format(
            best_epoch,
            train_loss,train_acc,
            valid_loss,valid_acc,valid_rmse,
            test_loss,test_acc,test_rmse
        ))

if __name__ == "__main__":

    # python3 cheetah.py --size 32 --model lstm --log 1 --epochs 200

    parser = argparse.ArgumentParser()
    parser.add_argument('--model',default="lstm")
    parser.add_argument('--log',default=1,type=int)
    parser.add_argument('--size',default=32,type=int)
    parser.add_argument('--epochs',default=200,type=int)
    parser.add_argument('--sparsity',default=0.0,type=float)
    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()


    traffic_data = ENSO_data()
    model = ENSO_model(model_type = args.model,model_size=args.size,sparsity_level=args.sparsity)


    model.fit(traffic_data,epochs=args.epochs,log_period=args.log)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
<ipython-input-7-eb65e19964ab>:161: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  self.fused_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(model_size)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
<ipython-input-7-eb65e19964ab>:192: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.y = tf.co

logit shape:  (None, None, 1)


ValueError: Cannot feed value of shape (32, 107, 19) for Tensor Placeholder:0, which has shape (None, None, 2)

In [8]:
import matplotlib.pyplot as plt

epochs = range(1, len(train mae)+1)

plt.plot(epochs, train_rmse, 'bo', label='Training RMSE')
plt.plot(epochs, test_rmse, 'r', label='Test RMSE')
plt.title('Training and Test MSE')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-8-f2547a7e68f7>, line 3)